## Deliverable 2. Create a Customer Travel Destinations Map.

In [67]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [38]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidty,Cloudiness,Wind Speed,Current Description
0,0,Mokolo,CM,10.7398,13.8019,83.43,19,99,4.63,overcast clouds
1,1,Marion,US,39.7684,-86.1555,57.20,50,75,6.91,broken clouds
2,2,Severo-Kurilsk,RU,50.6789,156.1250,35.19,96,100,27.98,moderate rain
3,3,Alta Floresta,BR,-9.8756,-56.0861,71.26,95,100,1.45,overcast clouds
4,4,Krasnoselkup,RU,65.7000,82.4667,-6.03,92,6,18.19,clear sky


In [41]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# get user input to suggest where user should travel based on preferences
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [45]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] < max_temp)]

preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidty,Cloudiness,Wind Speed,Current Description
1,1,Marion,US,39.7684,-86.1555,57.20,50,75,6.91,broken clouds
3,3,Alta Floresta,BR,-9.8756,-56.0861,71.26,95,100,1.45,overcast clouds
5,5,Otane,NZ,-39.8833,176.6333,68.00,59,92,4.00,overcast clouds
6,6,Atar,MR,20.5169,-13.0499,79.56,14,97,6.87,overcast clouds
10,10,Yellowknife,CA,62.4560,-114.3525,41.00,48,20,10.36,few clouds
11,11,Rikitea,PF,-23.1203,-134.9692,77.92,75,11,6.91,few clouds
12,12,Atambua,ID,-9.1061,124.8925,79.52,61,100,2.93,overcast clouds
13,13,Kapaa,US,22.0752,-159.3190,78.80,61,40,13.80,scattered clouds
14,14,Tura,IN,25.5198,90.2201,69.31,85,0,4.09,clear sky
18,18,Touros,BR,-5.1989,-35.4608,75.20,94,40,3.44,scattered clouds


In [46]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidty                0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [64]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
pref_cities_df_clean = preferred_cities_df.dropna()

In [68]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pref_cities_df_clean[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Marion,US,57.20,broken clouds,39.7684,-86.1555,NaN
3,Alta Floresta,BR,71.26,overcast clouds,-9.8756,-56.0861,NaN
5,Otane,NZ,68.00,overcast clouds,-39.8833,176.6333,NaN
6,Atar,MR,79.56,overcast clouds,20.5169,-13.0499,NaN
10,Yellowknife,CA,41.00,few clouds,62.4560,-114.3525,NaN
11,Rikitea,PF,77.92,few clouds,-23.1203,-134.9692,NaN
12,Atambua,ID,79.52,overcast clouds,-9.1061,124.8925,NaN
13,Kapaa,US,78.80,scattered clouds,22.0752,-159.3190,NaN
14,Tura,IN,69.31,clear sky,25.5198,90.2201,NaN
18,Touros,BR,75.20,scattered clouds,-5.1989,-35.4608,NaN


In [70]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # set lat and long as parameters for google maps search
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # try request
    try:
        # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("No hotels in this location... checking next city.")
        

No hotels in this location... checking next city.
No hotels in this location... checking next city.
No hotels in this location... checking next city.
No hotels in this location... checking next city.
No hotels in this location... checking next city.
No hotels in this location... checking next city.
No hotels in this location... checking next city.
No hotels in this location... checking next city.
No hotels in this location... checking next city.
No hotels in this location... checking next city.
No hotels in this location... checking next city.
No hotels in this location... checking next city.
No hotels in this location... checking next city.
No hotels in this location... checking next city.
No hotels in this location... checking next city.
No hotels in this location... checking next city.
No hotels in this location... checking next city.
No hotels in this location... checking next city.
No hotels in this location... checking next city.
No hotels in this location... checking next city.


In [74]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# could have done this by resetting the df to everything that did not equal blank - then would not have had to re run the API calls...

# clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != 0]

In [75]:
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [58]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [76]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [78]:
# 11a. Add a marker layer for each city to the map. 
# create gmap fig
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# add markers to figure
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))